![ModernHouseForSale](Images/Modern%20House%20for%20Sale.png)

**Authors**: 

**Date created**: 19-12-2023

**Discipline**: Data Science PT-05

## Overview

## Business Problem

## Data Understanding

In [37]:
# importing standard libraries

import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')


In [38]:
# Load and preview the dataset

data = pd.read_csv("Data/kc_house_data.csv")
data.head(5)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,NONE,...,7 Average,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,NO,NONE,...,7 Average,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,NO,NONE,...,6 Low Average,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,NO,NONE,...,7 Average,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,NO,NONE,...,8 Good,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


In [39]:
# viewing the metadata of our 'data' df

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     19221 non-null  object 
 9   view           21534 non-null  object 
 10  condition      21597 non-null  object 
 11  grade          21597 non-null  object 
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  object 
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   17755 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long  

In [40]:
# Exploring missing values, data types and dropping irrelevant columns

data.isna().sum()

id                  0
date                0
price               0
bedrooms            0
bathrooms           0
sqft_living         0
sqft_lot            0
floors              0
waterfront       2376
view               63
condition           0
grade               0
sqft_above          0
sqft_basement       0
yr_built            0
yr_renovated     3842
zipcode             0
lat                 0
long                0
sqft_living15       0
sqft_lot15          0
dtype: int64

In [41]:
data.view.unique()

array(['NONE', nan, 'GOOD', 'EXCELLENT', 'AVERAGE', 'FAIR'], dtype=object)

In [42]:
# View a sample of homes with null values for 'view'

null_views = data[data['view'].isna()]
null_views.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
7,2008000270,1/15/2015,291850.0,3,1.50,1060,9711,1.0,NO,NaN,...,7 Average,1060,0.0,1963,0.0,98198,47.4095,-122.315,1650,9711
114,8961960160,10/28/2014,480000.0,4,2.50,3230,16171,2.0,NO,NaN,...,9 Better,2520,710.0,2001,0.0,98001,47.3183,-122.253,2640,8517
129,7853210060,4/6/2015,430000.0,4,2.50,2070,4310,2.0,NO,NaN,...,7 Average,2070,0.0,2004,0.0,98065,47.5319,-121.850,1970,3748
205,3456000310,8/4/2014,840000.0,4,1.75,2480,11010,1.0,NO,NaN,...,9 Better,1630,850.0,1966,NaN,98040,47.5378,-122.219,2770,10744
487,1895000260,7/21/2014,207950.0,2,2.00,890,5000,1.0,NO,NaN,...,6 Low Average,890,0.0,1917,0.0,98118,47.5158,-122.264,1860,5000


In [43]:
# Viewing coordinates on OpenStreetMap for Houses with null views

import folium
from folium.plugins import MarkerCluster

# Getting the average latitude

average_latitude = null_views.lat.mean()

# Getting average longitude

average_longitude = null_views.long.mean()

# Create a folium map centered around the average location
mymap = folium.Map(location=[average_latitude, average_longitude], zoom_start=12)

# Adding markers 

marker_cluster = MarkerCluster().add_to(mymap)

for index, row in null_views.iterrows():
    folium.Marker([row['lat'], row['long']], popup=f"House {index}").add_to(marker_cluster)


# display the map

mymap


In [44]:
# Checking the same for null values for Waterfront Column

null_waterfront = data[data['waterfront'].isna()]

# Getting the average latitude

average_latitude = null_waterfront.lat.mean()

# Getting average longitude

average_longitude = null_waterfront.long.mean()

# Create a folium map centered around the average location
mymap = folium.Map(location=[average_latitude, average_longitude], zoom_start=12)

# Adding markers 

marker_cluster = MarkerCluster().add_to(mymap)

for index, row in null_waterfront.iterrows():
    folium.Marker([row['lat'], row['long']], popup=f"House {index}").add_to(marker_cluster)


# display the map

mymap


Using a combination of the above interactive maps and sampling a few houses on Google Street View, it is apparent that
a vast majority of these units do not have a notable view/ waterfront. This suggests that the null values can be filled in accordingly. The assumption here is that it will not affect our analysis adversely.

In [45]:
# Filling missing values in the 'view' and 'waterfront' columns

data['waterfront'].fillna('NO', inplace=True)
data['view'].fillna('NONE', inplace=True)

# verifying the operation was successful

data.isna().sum()

id                  0
date                0
price               0
bedrooms            0
bathrooms           0
sqft_living         0
sqft_lot            0
floors              0
waterfront          0
view                0
condition           0
grade               0
sqft_above          0
sqft_basement       0
yr_built            0
yr_renovated     3842
zipcode             0
lat                 0
long                0
sqft_living15       0
sqft_lot15          0
dtype: int64

In [46]:
data.yr_renovated.value_counts()

0.0       17011
2014.0       73
2003.0       31
2013.0       31
2007.0       30
          ...  
1946.0        1
1959.0        1
1971.0        1
1951.0        1
1954.0        1
Name: yr_renovated, Length: 70, dtype: int64

In [47]:
data.yr_renovated.isna().sum()

3842

This particular column exhibits 3,842 missing values. Moreover, 17,011 entries indicate a 'yr_renovated' value of 0, presumably signifying that the residence underwent no renovation. Combined, these instances represent over 96% of the dataset. Instead of imputing these values with arbitrary numbers, a more informative approach would be to create a new column indicating whether the property underwent renovation in the decade preceding its sale.

In [48]:
yr_renovated = data["yr_renovated"]
yr_sold = data['date'].str[-4:].astype(int)
data['renovated_last_10'] = (yr_renovated >= (yr_sold - 10))
data['renovated_last_10'] = data['renovated_last_10'].map({True: 'Yes', False: 'No'})
# Keeping this as a string for now in order to group it with categorical variables later.
data.head(5)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,renovated_last_10
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NO,NONE,...,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650,No
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,NO,NONE,...,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639,No
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,NO,NONE,...,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062,No
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,NO,NONE,...,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000,No
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,NO,NONE,...,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503,No


In [49]:
# dropping the 'yr_renovated' column from our DataFrame

data.drop('yr_renovated', axis=1, inplace=True)

In [52]:
# Verifying the column was successfully dropped

print(data.head(5))

# Confirming no missing values are left in any of our columns

print(data.isna().sum())

           id        date     price  bedrooms  bathrooms  sqft_living  \
0  7129300520  10/13/2014  221900.0         3       1.00         1180   
1  6414100192   12/9/2014  538000.0         3       2.25         2570   
2  5631500400   2/25/2015  180000.0         2       1.00          770   
3  2487200875   12/9/2014  604000.0         4       3.00         1960   
4  1954400510   2/18/2015  510000.0         3       2.00         1680   

   sqft_lot  floors waterfront  view  ...          grade sqft_above  \
0      5650     1.0         NO  NONE  ...      7 Average       1180   
1      7242     2.0         NO  NONE  ...      7 Average       2170   
2     10000     1.0         NO  NONE  ...  6 Low Average        770   
3      5000     1.0         NO  NONE  ...      7 Average       1050   
4      8080     1.0         NO  NONE  ...         8 Good       1680   

   sqft_basement yr_built  zipcode      lat     long  sqft_living15  \
0            0.0     1955    98178  47.5112 -122.257           